In [ ]:
%pip install langchain-openai langchain-core pydantic python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal
from pydantic import BaseModel, Field

load_dotenv()

True

In [3]:
model = ChatOpenAI(
    model="solar-pro2",
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)
parser = StrOutputParser()

In [4]:
# --- [원본 Cell 3: 간단한 체인] ---
print("--- [Cell 3: Simple Chain (model | parser)] ---")
chain = model | parser
messages_cell1 = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]
print(chain.invoke(messages_cell1))

--- [Cell 3: Simple Chain (model | parser)] ---
(아, 개스톤이라니... 그의 목소리부터 이미 동네 술집 분위기를 풍기네요. 미소는 짓지만 손은 살짝 긴장되어 테이블보를 만지작거려요.)  

"오, 개스톤! 오늘 저녁이라니... (웃음을 참으며) 당신답지 않은 제안인데? 혹시 마을에 새로운 술집이라도 생겼나요?" (잔뜩 기대에 찬 눈으로 바라보지만, 속으로는 '술집에서 또 사냥 이야기 늘어놓겠지' 생각하며 한숨을 삼킵니다.)  

"하지만 오늘 밤은 좀 특별한 계획이 있어요. 아버지가 고장 난 시계를 고치러 오셨는데, 야수... 아니, 집사와 함께 저녁 식사를 하기로 했어요." (말끝을 흐리며 창밖을 바라봅니다. 야수라는 단어가 튀어나올 뻔했죠.)  

"다음에 기회가 되면 꼭! (손짓으로 강조하는 척하며 속삭입니다.) 단, 이번에는 당신이 요리를 해보는 건 어때요? '사냥감 잡기' 대신 '계란 삶기'라도 도전해보시구요!" (농담처럼 말하지만, 사실은 진지한 제안이에요. 그가 주방 근처에도 못 오게 하려고.)  

(말투는 여전히 밝지만, 눈빛은 '절대로 내 성에 초대하지 않을 거야'라고 경고합니다.)  

*미녀는 개스톤의 허풍을 잘 알지만, 진정한 우정은 진심을 나누는 것이라 믿는답니다.*


In [5]:
# --- [원본 Cell 4, 5: 프롬프트 템플릿 체인] ---
print("\n--- [Cell 5: Prompt + Model + Parser Chain] ---")
system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"


--- [Cell 5: Prompt + Model + Parser Chain] ---


In [6]:
# (원본 Cell 4)
prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

In [7]:
# (원본 Cell 5)
chain = prompt_template | model | parser

print(chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
}))

오, 야수님! 정말 깜짝 놀랐어요. 이렇게 직접 말씀하시다니… (살짝 당황했지만 미소를 지으며) 오늘 저녁은 제가 준비한 특별 요리가 있는데, 함께해 주신다면 정말 영광이에요.  

(창을 통해 비치는 석양을 바라보며) 저녁 노을이 너무 아름다운 날이에요. 정원의 장미도 오늘따라 유난히 붉네요… (조심스럽게 눈빛을 맞추며) 물론, 제가 만든 수프를 드시고 목이 메어도 책임지지 않아요. 농담이에요! (웃음)  

어떻게 생각하시나요? 아직 성의 문을 닫을 시간은 남았어요.  

*P.S. 혹시… 빨간 드레스 입고 가도 될까요?* 🌹


In [8]:
# --- [원본 Cell 7: Pydantic (Structured Output) 체인] ---
print("\n--- [Cell 7: Structured Output Chain] ---")
class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

# .with_structured_output은 OpenAI/Upstage 모델에서 잘 작동합니다. (원본과 동일)
structured_llm = model.with_structured_output(Adlib)
adlib_chain = prompt_template | structured_llm

print(adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
}))


--- [Cell 7: Structured Output Chain] ---
answer='오, 개스톤 씨. 저녁 초대는 정말 친절하시지만... 오늘은 좀 바빠서요. 다음에 기회가 된다면 꼭 함께 해요!' main_emotion='중립' main_emotion_intensity=0.6
